# Parallel KMeans implementation
Based on J. Y. Q. H. Z. W. a. J. C. Bowen Wang, “Parallelizing K-means-based Clustering on Spark,” International Conference on Advanced Cloud and Big Data, 2016. 

## Parallel partition-based algorithm outline
1. Initialize centroids by randomly selecting k points from the data set. Broadcast selected centroids to all nodes
1. While centroids are moving:
    1. Broadcast current centroids
    1. For each point
        1. Compute the distance to all centroids
        1. Asign the closest cluster
    1. For each cluster
        1. Compute local mean
    1. Compute the mean for each cluster for each partition
 

## Adaptations made to suggested implementation of the algorithm:
1. The authors suggest using SparseVector, with chosen data sets it is better to use regular arrays
1. We use the random sample for centroids initialization as described in *Scalable K-Means++* because the quality of initial centroids has a major effect on the quality
1. We use crisp clustering only i. e. each point can be a member of one cluster only
1. We use Euclidian data because it is the recommended distance function for dense data.

## Description of production cluster on Azure:
We are using Azure HDIsight in order to run a spark cluster. We are using a cluster with the following configuration:
* two master nodes Standard_D12_V2 4 CPU Cores 28GB RAM
* eight slave nodes Standard_D13_V2 8 CPU Cores 56GB RAM


In [1]:
import os
import requests
import time
from pprint import pprint

from numpy import array
import numpy as np
import pandas as pd
from itertools import groupby

In [2]:
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import split, col, size, trim, lit
from pyspark.ml.linalg import Vectors, DenseVector

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("FirstSparkJob") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/29 14:54:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/29 14:54:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
RANDOM_SEED = 301191
K = 50 

In [4]:
A3_DATASET_URL = "https://cs.joensuu.fi/sipu/datasets/a3.txt"
DATA_FOLDER = "/home/jovyan/work/data"
A3_LOCAL_PATH = os.path.join(DATA_FOLDER, "a3.txt")

In [5]:
# Download Data
response = requests.get(A3_DATASET_URL)
if not os.path.exists(A3_LOCAL_PATH):
    with open(A3_LOCAL_PATH, 'wb') as file:
        file.write(response.content)

In [6]:
# Load clean data into spark
data = sc.textFile(A3_LOCAL_PATH)
parsed_data = data.map(lambda row: Vectors.dense([float(x) for x in row.strip().split()]))
parsed_data.cache()

parsed_data.take(5)

[DenseVector([53920.0, 42968.0]),
 DenseVector([52019.0, 42206.0]),
 DenseVector([52570.0, 42476.0]),
 DenseVector([54220.0, 42081.0]),
 DenseVector([54268.0, 43420.0])]

In [7]:
def compute_distance(p, centroid):
    return np.sqrt(np.sum((np.array(p) - np.array(centroid))**2))

def kmeans(rdd, centroids, max_iters=20, tolerance=0.1):
    for i in range(max_iters):
        timer = time.time()
        
        # Broadcast the centroids to all nodes
        broadcast_centroids = spark.sparkContext.broadcast(centroids)
        clustered_rdd = rdd.map(lambda p: (np.argmin([compute_distance(p, c) for c in broadcast_centroids.value]), (p, 1)))

        # Recompute centroids by averaging points in each cluster
        new_centroids = (
            clustered_rdd
            .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))  # Sum points and count
            .map(lambda x: (x[0], x[1][0] / x[1][1]))  # Compute new centroids
            .collectAsMap()
        )

        new_centroids_arr = np.array([new_centroids[j] for j in range(len(centroids))], dtype=np.float64)
        
        # Calculate sum of shifts for each centroid
        shifts = np.sum(np.linalg.norm(new_centroids_arr - centroids, axis=1))
        
        print(f"Iteration:{i}\tShifts:{shifts}\ttime taken:{time.time()-timer}")
        if shifts < tolerance:
            break

        # Update the centroids
        centroids = new_centroids_arr
    
        # Free memory
        broadcast_centroids.unpersist()

    return centroids

# Run the K-Means with broadcasting
centroids = np.array(parsed_data.takeSample(False, K, seed=RANDOM_SEED), dtype=np.float64)
final_centroids = kmeans(parsed_data, centroids)
print("Final centroids:", final_centroids)

Iteration:0	Shifts:99201.44807107747	time taken:1.3361728191375732


Iteration:1	Shifts:54617.94960414767	time taken:1.193866491317749


Iteration:2	Shifts:40523.517798017914	time taken:1.2035140991210938


Iteration:3	Shifts:25362.289023665504	time taken:1.172621488571167


Iteration:4	Shifts:11000.387704176874	time taken:1.1952300071716309


...

Iteration:14	Shifts:149.09461400727525	time taken:1.190490484237671


Iteration:15	Shifts:125.7231844690503	time taken:1.239217758178711


Iteration:16	Shifts:70.56990347862424	time taken:1.2039122581481934


Iteration:17	Shifts:53.95295020035813	time taken:1.1634669303894043


Iteration:18	Shifts:0.0	time taken:1.146303653717041
Final centroids: [[59542.68604651 17758.12209302]
 [13925.54966887 43347.90066225]
 [ 7088.34666667 11170.17333333]
 [56084.33557047 35245.77852349]
 [ 5162.75838926 42433.63758389]
 [23203.99324324 44859.15540541]
 [10289.22       16705.62666667]
 [43257.42384106  4639.56953642]
 [30480.4527027  45020.29054054]
 [57887.9        59083.57333333]
 [16418.82954545 52911.36363636]
 [ 9635.12080537 29483.69127517]
 [19690.73426573 60544.46853147]
 [27408.41891892 57946.13513514]
 [17530.36       25319.5       ]
 [49874.3718593  23959.46733668]
 [26011.1        55383.26      ]
 [57649.97378277 10902.67790262]
 [47111.77419355 59097.11612903]
 [25553.4        28055.64666667]
 [30520.06722689 60468.28571429]
 [43229.93055556 54920.54861111]
 [42100.8        27942.968     ]
 [ 4725.37162162 54020.82432432]
 [ 4099.93377483 23940.06622517]
 [48201.19375    10293.525     ]
 [50913.33986928 48696.47058824]
 [10207.33552632 50195.21710526]
 [3702

In [11]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

centroid_to_point = parsed_data.map(
    lambda point: (
        np.argmin([np.linalg.norm(point - c) for c in final_centroids]), # cluster_id
        point # original point
    )
).collect()

cluster_labels, data = zip(*centroid_to_point)
silhouette = silhouette_score(data, cluster_labels)
silhouette

np.float64(0.5596370095167552)